In [60]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

These are the parameters for the query.  The "place_id" corresponds to DuPage Forest Preserves as defined in iNaturalist

In [3]:
DFP_PlaceID = 159205
iconic_taxa = "Plantae"
d1="2017-05-01"

Query quality_grade=research&identifications=any&iconic_taxa[]=Plantae&place_id=159205&d1=2017-05-01&d2=2022-05-17 Columns id, observed_on_string, observed_on, quality_grade, description, num_identification_agreements, num_identification_disagreements, oauth_application_id, place_guess, latitude, longitude, positioning_device, species_guess, scientific_name, common_name, iconic_taxon_name, taxon_id

In [58]:
query=f"https://api.inaturalist.org/v1/observations?place_id=159205&d1=2017-05-01&iconic_taxa=Plantae&quality_grade=research&order=desc&order_by=created_at&per_page=200&page="

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

Initialize some control values

In [5]:
page=1
results = [1,2]

In [6]:
reclist = []
while len(results) > 0:
    res = requests.get(query+f"{page}")
    if res.status_code == 200:
        js = res.json()
        results = js["results"]
        for obs in results:
            this = {}
            if obs['taxon']['rank'] == "species":
                this['SciName'] = obs['taxon']['name']
                this['common_name'] = obs['taxon']['preferred_common_name']
            this['longitude'] = obs['geojson']['coordinates'][0]
            this['latitude'] =  obs['geojson']['coordinates'][1]
            this['observed_on'] = obs['observed_on']
            this['who'] = obs['user']['login']
            reclist.append(this)
    page += 1

In [9]:
df_inat = pd.DataFrame(reclist)
df_inat.to_excel('inat_raw_observations.xlsx')

Can use saved observations instead of re-querying iNaturalist

In [ ]:
df_inat = pd.read_excel('inat_raw_observations.xlsx')

In [116]:
in_species = set(df_inat['SciName'].unique())

now get the species observed by The Forest Preserve

In [117]:
fp_df = pd.read_excel("DMP Flora by EcoUnit.xlsx")

In [118]:
in_species  |= set(fp_df["SciName"])

In [26]:
def get_lb_data_from_soup(soup, sn, symbol):
    #print('Getting Lady Bird data from soup')
    rec = {"SciName": sn, 
           "Symbol": symbol, 
           "Bloom Time:": " ",
           "Jan": 0,
           "Feb": 0,
           "Mar": 0,
           "Apr": 0,
           "May": 0,
           "Jun": 0,
           "Jul": 0,
           "Aug": 0,
           "Sep": 0,
           "Oct": 0,
           "Nov": 0,
           "Dec": 0,
           "Special Value to Native Bees": 0 ,
           "Special Value to Bumble Bees": 0, 
           "Nectar Source:": 0, 
           "Larval Host:": "No",
           "Larval Host(Monarch)": 0,
           "Larval Host(Other)": 0}
    h4s = soup.find_all("h4")
    #print("h4's found: ",h4s)
    for n in h4s:
        #print('\n\n', '####', n, n.string)
        if n.string == "Bloom Information":
            #print("Found Bloom Information")
            sibs = n.find_next_siblings()
            for s in sibs:
                #print(s)
                if s.string == "Bloom Time:":
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    mos = s.next_sibling.split(",")
                    for m in mos:
                        rec[m.strip()] = 1                    
                    rec[s.string] = s.next_sibling
        if n.string == "Benefit":
            sibs = n.find_next_siblings()
            for s in sibs:
                #print(s)
                if s.string == "Nectar Source:":
                    print("found benefit for ", sn, symbol)
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    rec[s.string] = 1
                if s.string == "Larval Host:":
                    print("found benefit for ", sn, symbol)
                    #print(n.parent)
                    #print(s)
                    #print(s.next_sibling)
                    rec[s.string] = s.next_sibling
                    if "onarch" in s.next_sibling:
                        rec["Larval Host(Monarch)"] = 1
                    else: 
                        rec["Larval Host(Other)"] = 1
        if n.string == "Value to Beneficial Insects":
            #print(n.parent.prettify())
            for s in n.find_next_siblings():
                #print(s)
                if s.string: 
                    if (s.string.strip().startswith("Special") and (s.string.find("Honey") == -1)):
                        #print('adding string', s)
                        rec[s.string.replace("\xa0", " ")] = 1
                #print(s.next_sibling)
                if (str(s.next_sibling).strip().startswith("Special") and (str(s.next_sibling).find("Honey") == -1)):
                    #print("adding next sibling", s.next_sibling)
                    rec[str(s.next_sibling.strip().replace("\xa0", " "))] = 1                
    return rec

In [62]:
def get_lb_page_for_sci_name(sn):
    try:
        symbol = lookup_list.loc[[sn]]["Symbol"].values[0]
    except KeyError:
        return("", "", "")
    url_template = f"https://www.wildflower.org/plants/result.php?id_plant={symbol}"
    #print(url_template)
    res = requests.get(url_template, headers=headers)
    if res.status_code != 200:
        sys.exit("page request failed")
    soup = BeautifulSoup(res.text, 'lxml')
    return (soup, sn, symbol)
 

Run the two above functions once for each species reported, and collect the records into a dataframe

In [30]:
plup = pd.read_csv("usda_plant_symbols.csv")

In [31]:
plup["SciName"] =plup.apply(lambda r: " ".join(r['Scientific Name with Author'].split()[:2]), axis=1)  
plu = plup.drop_duplicates(subset=["SciName"]).set_index('SciName')

In [50]:
in_df = pd.DataFrame(in_species, columns=["SciName"]).dropna()
lookup_list = (in_df
               .join(plu,on="SciName")[["SciName", "Symbol"]]
               .dropna()
               .set_index("SciName") )

In [63]:
rec_list = []
for plant in in_df["SciName"]:
    #print(plant)
    page, sn, symbol = get_lb_page_for_sci_name(plant)
    if symbol != "":
        rec_list.append(get_lb_data_from_soup(page, sn, symbol))


found benefit for  Carpinus caroliniana CACA18
found benefit for  Poa pratensis POPR
found benefit for  Sassafras albidum SAAL5
found benefit for  Cephalanthus occidentalis CEOC2
found benefit for  Aristida oligantha AROL
found benefit for  Salix discolor SADI
found benefit for  Gleditsia triacanthos GLTR
found benefit for  Celtis occidentalis CEOC
found benefit for  Fragaria virginiana FRVI
found benefit for  Chelone glabra CHGL2
found benefit for  Chasmanthium latifolium CHLA5
found benefit for  Fraxinus pennsylvanica FRPE
found benefit for  Morus rubra MORU2
found benefit for  Viburnum lentago VILE
found benefit for  Cornus sericea COSE16
found benefit for  Cornus florida COFL2
found benefit for  Fraxinus lanceolata FRPE
found benefit for  Leersia oryzoides LEOR
found benefit for  Amaranthus tuberculatus AMTU
found benefit for  Penstemon digitalis PEDI
found benefit for  Dalea candida DACA7
found benefit for  Astragalus canadensis ASCA11
found benefit for  Verbena simplex VESI
found

In [64]:
plant_df = pd.DataFrame(rec_list)

Save the species information to an Excel sheet

In [65]:
plant_df.to_excel("LadyBirdData_master.xlsx")

Can use the saved plant info data if it seems current enough

In [ ]:
plant_df = pd.read_excel("LadyBirdData_master.xlsx")

Clean up columns

In [66]:
plant_df = plant_df[['SciName', 'Symbol', 'Bloom Time:', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
       'Special Value to Native Bees', 'Special Value to Bumble Bees',
       'Nectar Source:', 'Larval Host:', 'Larval Host(Monarch)',
       'Larval Host(Other)']]

In [67]:
# Open the shapefile
ecoUnits = gpd.GeoDataFrame.from_file('./Eco Unit Shapefile/')

make a geodataframe from the observations list (the inat has lat/long of observation)
Make  the coordinate reference systems match

Note that 55 iNat observations do not have a species ID becasue the initial observer did not know or was wrong.  (Later identifiers agreed on the species)  I dropped these for simplicity and because 55 of 8228 seemed not worth hunting down the later ids.

In [76]:
gdf_inats = gpd.GeoDataFrame(df_inat,
    geometry=gpd.points_from_xy(df_inat.longitude, df_inat.latitude))
gdf_inats = gdf_inats[['SciName', 'common_name', 'longitude', 'latitude','observed_on', 'who', 'geometry']].dropna()
gdf_inats = gdf_inats.set_crs(crs="EPSG:4326")

In [77]:
eco_lats = ecoUnits.to_crs("EPSG:4326")

Do a spatial join to match each observation to the EUnit it was made in.  Drop multilple observations of the same plant in the same EUnit

In [122]:
mdf = eco_lats.sjoin(gdf_inats, how="left", predicate="contains").sort_values("EUnit")

Add the plant characteristics to the observations

In [125]:
mdf =  mdf.join(plant_df.reset_index().set_index("SciName"), on="SciName")

Only count a plant once in each EUnit, summarize a bit

In [127]:
mdf = mdf.drop_duplicates(subset=["EUnit", "SciName"])
mdf["habitat_index"] = (mdf["Special Value to Native Bees"] 
     + mdf["Special Value to Bumble Bees"] 
     + mdf["Nectar Source:"] 
     + mdf["Larval Host(Monarch)"] 
     + mdf["Larval Host(Other)"])

mdf["# plant-months with blooms"] = (mdf["Jan"] + mdf["Feb"] + mdf["Mar"] + mdf["Apr"] 
     + mdf["May"] + mdf["Jun"] + mdf["Jul"] + mdf["Aug"] + mdf["Sep"] 
     + mdf["Oct"] + mdf["Nov"] + mdf["Dec"] )

int_df = mdf[mdf.habitat_index > 0]

In [129]:
sum_df = mdf.groupby([ 'EcosName', 'EUnit' ]).agg({'SciName':'count', 
                     'Special Value to Native Bees':'sum', 
                     'Special Value to Bumble Bees':'sum',
                     'Nectar Source:': "sum", 
                     'Larval Host:': "sum",
                     'Larval Host(Monarch)': "sum", 
                     'Larval Host(Other)': "sum",
                     '# plant-months with blooms': "sum",
                     'habitat_index':'sum'
})

In [ ]:
sum_df

In [83]:
with pd.ExcelWriter("inat_interesting_units.xlsx") as writer:
    sum_df.to_excel(writer, sheet_name="SummaryData")
    mdf.set_index("EUnit").to_excel(writer, sheet_name="DetailData")

Now do the forest preserve observations

In [133]:
fp_df =  fp_df.join(plant_df.reset_index().set_index("SciName"), on="SciName")
fp_df = fp_df.rename(columns={"EcosUnit": "EUnit"})

fp_df = fp_df.drop_duplicates(subset=["EUnit", "SciName"])
fp_df["habitat_index"] = (fp_df["Special Value to Native Bees"] 
     + fp_df["Special Value to Bumble Bees"] 
     + fp_df["Nectar Source:"] 
     + fp_df["Larval Host(Monarch)"] 
     + fp_df["Larval Host(Other)"])

fp_df["# plant-months with blooms"] = (fp_df["Jan"] + fp_df["Feb"] + fp_df["Mar"] + fp_df["Apr"] 
     + fp_df["May"] + fp_df["Jun"] + fp_df["Jul"] + fp_df["Aug"] + fp_df["Sep"] 
     + fp_df["Oct"] + fp_df["Nov"] + fp_df["Dec"] )

int2_df = fp_df[fp_df.habitat_index > 0]

In [154]:
sum2_df = fp_df.groupby([ 'EcosName', 'EUnit' ]).agg({'SciName':'count', 
                     'Special Value to Native Bees':'sum', 
                     'Special Value to Bumble Bees':'sum',
                     'Nectar Source:': "sum", 
                     'Larval Host:': "sum",
                     'Larval Host(Monarch)': "sum", 
                     'Larval Host(Other)': "sum",
                     '# plant-months with blooms': "sum",
                     'habitat_index':'sum'
})

In [137]:
with pd.ExcelWriter("fp_interesting_units.xlsx") as writer:
    sum2_df.to_excel(writer, sheet_name="SummaryData")
    fp_df.set_index("EUnit").to_excel(writer, sheet_name="DetailData")

The combined data

In [138]:
fp2_df = fp_df.drop(columns="PresName")

In [144]:
com_df = pd.concat([sum2_df, sum_df], keys=["FP ", "iNAT "]).reset_index().sort_values("EUnit")

In [157]:
com_sum_df = com_df.groupby(["EUnit"]).agg({"level_0": "sum",
                     'SciName':'count', 
                     'Special Value to Native Bees':'sum', 
                     'Special Value to Bumble Bees':'sum',
                     'Nectar Source:': "sum", 
                     'Larval Host(Monarch)': "sum", 
                     'Larval Host(Other)': "sum",
                     '# plant-months with blooms': "sum",
                     'habitat_index':'sum'
})

In [160]:
com_sum_df.to_excel("combined_summary.xlsx")

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format